In [1]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
#monkey.patch_all(thread=False,select=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = Trueee
server.serve_forever()
#http://127.0.0.2:5000/

127.0.0.1 - - [2021-07-15 10:03:29] "GET /upload_jobs HTTP/1.1" 200 7812 0.031912
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57471 0.004068
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/css/template.css HTTP/1.1" 200 16699 0.002147
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/css/mycss.css HTTP/1.1" 200 2488 0.002303
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88448 0.002989
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21563 0.003997
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60317 0.002357
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/js/template.js HTTP/1.1" 200 1713 0.003078
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/vendor/fontawesome-free/webfonts/fa-solid-900.woff2 HTTP/1.1" 200 76372 0.005068
127.0.0.1 - - [2021-07-15 10:03:29] "GET /static/vendor

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
%%time
from sys import path
from datetime import datetime
import os
path.append(os.path.join(os.getcwd(),'src'))

from commons import convert_file_size,init_db_manager
import pandas as pd
from jobs import *
#a=job_world_fond_indexes(init_db_manager())

In [ ]:
a=job_sectors(init_db_manager())

In [ ]:
db=init_db_manager()
a=db.get_table('sec_sector',result='json')
#a=job_update_sec_hist(init_db_manager())

In [ ]:
for row in db.get_table_cursor('sec_sector',query={'ticker': ['IRKT','RKKE']}):
    print(list(row.values()))

In [ ]:
for row in db.db['sec_sector'].find({'ticker':{'$in': ['IRKT','RKKE']}}):
    print(row)

In [ ]:
db.get_table('sec_sector',query={'ticker':{'$in': ['IRKT','RKKE']}},dict_key="ticker",result='dict',columns=['action',
                                                                                                             'industry',
                                                                                                             'sector','ticker'])


In [ ]:
def f(c=11,a=3,b=6):
    return a/b+c
aa={'a':5,'b':15}
for k,v in aa.items():
    print(k,v)

In [ ]:
def __get_table(table_name,result='matrix',view_id=None,params=None,post_data={},query:dict={},dict_key=None,columns=None):
    kvargs={'result':result,'query':query,"dict_key":dict_key,'columns':columns}
    for k,v in post_data.items():
        kvargs[k]=v
    mm=mongo_manager(config)
    if view_id is not None:
        return __get_view(mm,view_id,result,params=params,)
    else:
        return mm.get_table(table_name,**kvargs)

In [ ]:
list(row.values())

In [ ]:
db.find_one('sec_sector',{'ticker': ['IRKT']})

In [ ]:
a=pd.DataFrame(a)
a

In [ ]:
a.shape

In [ ]:
db.get_table('sec_history_manager_mosex')

In [ ]:
db.get_table('mosex_sec_history')[-10:]

In [ ]:
r=db.find_one('sec_history_manager_mosex',{"secid":'GAZP111'})

In [ ]:
from datetime import datetime
datetime.strptime('2016-01-04','%Y-%m-%d').month

In [ ]:
db=init_db_manager()

In [ ]:
df[df.boardid=='TQBR'].tradedate.nunique()

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

from external_sources import mosex,tradingview
import pandas as pd
a=mosex()
#a=tradingview()

In [ ]:

aa=a.security_hist('SBER')

In [ ]:
aa[1][1]

In [ ]:
datetime.now()

In [ ]:
for el in list(map(lambda s:s.lower(),aa[1][0])):
    print(el+"="+el+"_ind,")

In [ ]:
aa[1][0]

In [ ]:
df=aa[1][1]

In [ ]:
aa.json()['history.cursor']

In [ ]:
%%time
a=job_sectors(init_db_manager())

In [ ]:
from requests import Session
from multiprocessing.dummy import Pool as ThreadPool
s = Session()
s.get("http://ya.ru")
def f(url):
    return s.get(url)


    
urls=[r'https://www.rbc.ru/',r'http://ya.ru/']

In [ ]:
number_of_years=6
month_in_year=12
total_month=number_of_years*month_in_year

In [ ]:
function around_year(number_of_years,month_in_year){return number_of_years*month_in_year}
around_year(5,2)